## Part 1 - Produce Data
We elected to use the datagen connector to generate fake data for this assignment. The topic we used was 'stocktrades. The steps were as follows:
*  Open a browser and go to http://localhost:9021/
*  Select the available cluster
*  On the menu bar, select Connect
*  Click on the connect-default cluster in the Connect Clusters list.
*  Click on Add connector
*  Select DatagenConnector
*  Enter connector_stock_trades in the Name field

Then:
Generate a data stream with following configurations:
```
Key converter class: org.apache.kafka.connect.storage.StringConverter
kafka.topic: stocktrades
max.interval: 100
quickstart: Stock_Trades
```


## Part 2 - Using Ksql to create at least 2 streams with filtering from topics